In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial import cKDTree
from tqdm import tqdm
import gc
import os

# ============================================================
# Utility
# ============================================================

def downcast_float(df, dtype='float32'):
    """Convert numeric columns to float32 or float64."""
    if dtype not in ['float32', 'float64']:
        raise ValueError("dtype must be 'float32' or 'float64'")
    num_cols = df.select_dtypes(include=['float64', 'float32']).columns
    df[num_cols] = df[num_cols].astype(dtype)
    return df

### Player Metadata

The following derived features are calculated:

$$
\begin{align}
\text{player\_height\_in} &= 12h + i \\
\text{player\_age} &= \frac{(2023{-}01{-}01) - \text{birth\_date}}{365.25}
\end{align}
$$

Where:
- $h$ = height in feet  
- $i$ = height in inches  

These features capture physical and experience-related differences across players.


In [ ]:
# ============================================================
# Player Metadata
# ============================================================
def add_player_metadata(df):
    print('Computing metadata')
    df = df.copy()
    df['player_height_in'] = df['player_height'].apply(lambda x: int(x.split('-')[0])*12 + int(x.split('-')[1]))
    df['player_age'] = (pd.to_datetime('2023-01-01') - pd.to_datetime(df['player_birth_date'])).dt.days / 365.25
    return df

### Motion Dynamics

Let:

- $s$: speed (yards/second)  
- $a$: acceleration (yards/second^2)  
- $\theta$: movement direction (radians)  
- $m$: player mass (kilograms)

Then:

$$
\begin{align}
v_x &= (s + 0.5a\Delta t)\sin(\theta) \\
v_y &= (s + 0.5a\Delta t)\cos(\theta) \\
a_x &= a\sin(\theta) \\
a_y &= a\cos(\theta) \\
KE &= \frac{1}{2} m s^2
\end{align}
$$

These features capture **movement intention** and **energy state** of the player.


In [ ]:
# ============================================================
# Motion Dynamics
# ============================================================

def add_motion_features(df):
    print('Computing motion dynamics')
    df = df.copy()
    rad = np.deg2rad(df['dir'].fillna(0))
    dt = 0.1  # 10 fps
    df['velocity_x'] = (df['s'] + 0.5 * df['a'] * dt) * np.sin(rad)
    df['velocity_y'] = (df['s'] + 0.5 * df['a'] * dt) * np.cos(rad)
    df['accel_x'] = df['a'] * np.sin(rad)
    df['accel_y'] = df['a'] * np.cos(rad)
    mass_kg = df['player_weight'].fillna(200) / 2.20462
    df['kinetic_energy'] = 0.5 * mass_kg * (df['s'] ** 2)
    return df

## 🧭 Spatial Positioning Features

Given ball coordinates $(x_b, y_b)$ and player $(x, y)$:

$$
\begin{align}
d_{\text{ball}} &= \sqrt{(x_b - x)^2 + (y_b - y)^2} \\
\theta_{\text{ball}} &= \arctan\left(\frac{y_b - y}{x_b - x}\right)
\end{align}
$$

Directional closing speed:

$$
v_{\text{closing}} = v_x \frac{(x_b - x)}{d_{\text{ball}}} + v_y \frac{(y_b - y)}{d_{\text{ball}}}
$$


- **dist_from_sideline = min(y, 53.3 - y)**  
- **dist_from_endzone = min(x, 120 - x)**

Indicates spatial opportunity and threat range.

In [ ]:
# ============================================================
# 3️⃣ Spatial Positioning
# ============================================================

def add_spatial_features(df):
    print('Computing spatial features')
    if 'ball_land_x' not in df.columns:
        return df
    df = df.copy()
    dx = df['ball_land_x'] - df['x']
    dy = df['ball_land_y'] - df['y']
    df['distance_to_ball'] = np.sqrt(dx**2 + dy**2)
    df['angle_to_ball'] = np.arctan2(dy, dx)
    df['ball_dir_x'] = dx / (df['distance_to_ball'] + 1e-6)
    df['ball_dir_y'] = dy / (df['distance_to_ball'] + 1e-6)
    df['closing_speed'] = df['velocity_x'] * df['ball_dir_x'] + df['velocity_y'] * df['ball_dir_y']
    df['dist_from_sideline'] = np.minimum(df['y'], 53.3 - df['y'])
    df['dist_from_endzone'] = np.minimum(df['x'], 120 - df['x'])
    return df

## ⏱ 4. Temporal Progression

Let:

- $x_t, y_t$: player position at frame $t$ (yards)  
- $\Delta x_t, \Delta y_t$: displacement between consecutive frames $t-1$ and $t$ (yards)  
- $\text{frame\_speed}_t$: speed at frame $t$ (yards/second)  
- $\text{frame\_accel}_t$: acceleration at frame $t$ (yards/second²)  
- $t_{\text{norm}}$: normalized frame index

Then:

$$
\begin{align}
\Delta x_t &= x_t - x_{t-1} \\
\Delta y_t &= y_t - y_{t-1} \\
\text{frame\_speed}_t &= \sqrt{(\Delta x_t)^2 + (\Delta y_t)^2} \\
\text{frame\_accel}_t &= \text{frame\_speed}_t - \text{frame\_speed}_{t-1} \\
t_{\text{norm}} &= \frac{\text{frame index}}{N_{\text{frames}}}
\end{align}
$$

These features capture **motion evolution** and **speed changes** of the player within a play.


In [ ]:
# ============================================================
# 4️⃣ Temporal Progression
# ============================================================

def add_temporal_features(df):
    print('Computing temporal features')
    df = df.sort_values(['game_id','play_id','nfl_id','frame_id']).copy()
    g = ['game_id','play_id','nfl_id']
    df['dx'] = df.groupby(g)['x'].diff().fillna(0)
    df['dy'] = df.groupby(g)['y'].diff().fillna(0)
    df['frame_speed'] = np.sqrt(df['dx']**2 + df['dy']**2)
    df['frame_accel'] = df.groupby(g)['frame_speed'].diff().fillna(0)
    df['frame_norm_time'] = df.groupby(g).cumcount() / (df.groupby(g)['frame_id'].transform('count') + 1)
    return df

## 🤝 5. Interaction Features

Let:

- $p_i, v_i$: position and velocity of player $i$  
- $p_j, v_j$: position and velocity of opponent $j$  
- $d_{ij}$: distance between player $i$ and opponent $j$ (yards)  
- $v_{\text{closing}}$: closing speed between player $i$ and opponent $j$ (yards/second)  
- $\text{nearest\_opponent\_dist}$: distance to closest opponent  
- $\text{nearest\_teammate\_dist}$: distance to closest teammate (excluding self)  
- $\text{closing\_speed\_nearest\_opp}$: closing speed with nearest opponent

Then:

$$
\begin{align}
d_{ij} &= \|p_i - p_j\|_2 \\
v_{\text{closing}} &= - \frac{(v_i - v_j) \cdot (p_j - p_i)}{\|p_j - p_i\|_2} \\
\text{nearest\_opponent\_dist} &= \min_j d_{ij} \\
\text{nearest\_teammate\_dist} &= \min_{k \neq i} \|p_i - p_k\|_2 \\
\text{closing\_speed\_nearest\_opp} &= v_{\text{closing}} \text{ (for nearest opponent)}
\end{align}
$$

These features encode **pressure**, **separation**, and **reaction context** for the player.


In [ ]:
# ============================================================
# 5️⃣ Interaction (KDTree Optimized)
# ============================================================

def add_interaction_features(df, mode='fast'):
    print('Computing interections')
    df = df.copy()
    df['nearest_opponent_dist'] = np.nan
    df['nearest_teammate_dist'] = np.nan
    if mode == 'full':
        df['closing_speed_nearest_opp'] = np.nan

    for (gid, pid, fid), frame_df in tqdm(df.groupby(['game_id','play_id','frame_id']), 
                                          desc="Computing interactions", 
                                          leave=False):
        pos = frame_df[['x','y']].values
        vel = frame_df[['velocity_x','velocity_y']].values
        sides = frame_df['player_side'].values
        idxs = frame_df.index

        for team in ['Offense','Defense']:
            team_mask = sides == team
            opp_mask = ~team_mask
            if not opp_mask.any() or not team_mask.any():
                continue
            team_pos = pos[team_mask]
            opp_pos = pos[opp_mask]
            tree_opp = cKDTree(opp_pos)
            dist_opp, idx_opp = tree_opp.query(team_pos, k=1)
            df.loc[idxs[team_mask], 'nearest_opponent_dist'] = dist_opp

            if mode == 'full':
                rel_v = vel[team_mask] - vel[opp_mask][idx_opp]
                disp = opp_pos[idx_opp] - team_pos
                closing = -np.einsum('ij,ij->i', rel_v, disp) / (np.linalg.norm(disp, axis=1) + 1e-6)
                df.loc[idxs[team_mask], 'closing_speed_nearest_opp'] = closing

            if team_pos.shape[0] > 1:
                tree_team = cKDTree(team_pos)
                d_team, _ = tree_team.query(team_pos, k=2)
                df.loc[idxs[team_mask], 'nearest_teammate_dist'] = d_team[:,1]

        del pos, vel, sides
    return df

## 🛣 6. Route Features

Let:

- $dir_t$: movement direction of the player at frame $t$ (degrees)  
- $\Delta dir_t$: change in direction between consecutive frames  
- $cum\_turn_t$: cumulative turn up to frame $t$ (degrees)  
- $is\_breaking$: indicator if a sharp turn occurs  
- $route\_depth$: longitudinal distance from start ($x_t - x_0$)  
- $route\_width$: lateral distance from start ($y_t - y_0$)

Then:

$$
\begin{align}
\Delta dir_t &= dir_t - dir_{t-1} \\
cum\_turn_t &= \sum |\Delta dir_t| \\
is\_breaking &= 
\begin{cases} 
1, & \text{if } |\Delta dir_t| > 45^\circ \\ 
0, & \text{otherwise} 
\end{cases} \\
route\_depth &= x_t - x_0 \\
route\_width &= y_t - y_0
\end{align}
$$

These features capture **route curvature**, **break points**, and **receiver maneuvering**.


In [ ]:
# ============================================================
# Route / Coverage
# ============================================================

def add_route_features(df):
    print('Computing routes')
    df = df.sort_values(['game_id','play_id','nfl_id','frame_id']).copy()
    g = ['game_id','play_id','nfl_id']
    df['route_depth'] = df.groupby(g)['x'].transform(lambda x: x - x.iloc[0])
    df['route_width'] = df.groupby(g)['y'].transform(lambda y: y - y.iloc[0])
    df['dir_change'] = df.groupby(g)['dir'].diff().fillna(0)
    df['cum_turn'] = df.groupby(g)['dir_change'].transform(lambda x: x.abs().cumsum())
    df['is_breaking'] = (np.abs(df['dir_change']) > 45).astype(int)
    return df

In [ ]:
# ============================================================
# Core Feature Manager
# ============================================================

def add_selected_features(df, 
                          feature_groups=None, 
                          mode='fast', 
                          dtype='float32', 
                          chunked=False, 
                          save_dir=None):
    """
    Configurable feature generation with tqdm and saving.
    
    Args:
        df (pd.DataFrame): tracking dataframe
        feature_groups (list): e.g. ['player','motion','spatial']
        mode (str): 'fast' or 'full'
        dtype (str): 'float32' or 'float64'
        chunked (bool): process by game_id
        save_dir (str): optional output directory for parquet
    """
    if feature_groups is None:
        feature_groups = ['player','motion','spatial','temporal','interaction','route']
    
    os.makedirs(save_dir, exist_ok=True) if save_dir else None
    results = []

    if chunked:
        game_ids = df['game_id'].unique()
        for gid in tqdm(game_ids, desc="Processing games"):
            gdf = df[df['game_id'] == gid].copy()
            processed = _process_chunk(gdf, feature_groups, mode, dtype)
            if save_dir:
                path = os.path.join(save_dir, f"features_game_{gid}.parquet")
                processed.to_parquet(path, index=False)
            results.append(processed)
            del gdf, processed
            gc.collect()
        df = pd.concat(results, ignore_index=True)
    else:
        df = _process_chunk(df, feature_groups, mode, dtype)
        if save_dir:
            df.to_parquet(os.path.join(save_dir, "features_all.parquet"), index=False)

    print(f"Added {feature_groups} in {mode} mode ({dtype}) — total {len(df.columns)} cols")
    return df


def _process_chunk(df, groups, mode, dtype):
    if 'player' in groups: df = add_player_metadata(df)
    if 'motion' in groups: df = add_motion_features(df)
    if 'spatial' in groups: df = add_spatial_features(df)
    if 'temporal' in groups: df = add_temporal_features(df)
    if 'interaction' in groups: df = add_interaction_features(df, mode=mode)
    if 'route' in groups: df = add_route_features(df)
    df = downcast_float(df, dtype)
    gc.collect()
    return df

In [ ]:
from pathlib import Path
from glob import glob 

class Config:
    DATA_DIR = Path("/kaggle/input/nfl-big-data-bowl-2026-prediction/")
    
def load_input_files(path=Path(Config.DATA_DIR/'train')):
    print("loading data")
    input_files = sorted(glob(os.path.join(path, "input_2023_w*.csv")))
    output_files = sorted(glob(os.path.join(path, "output_2023_w*.csv")))
    dfs_input, dfs_output = [], []
    for f_in, f_out in zip(input_files, output_files):
        dfs_input.append(pd.read_csv(f_in))
        dfs_output.append(pd.read_csv(f_out))
    df_input = pd.concat(dfs_input, ignore_index=True)
    df_output = pd.concat(dfs_output, ignore_index=True)
    return df_input, df_output

df_input, df_output = load_input_files()
print("loading complete")

### Usage Examples
**✅ 1. Fast + float32**

In [ ]:
features = add_selected_features(df_input,
                                 feature_groups=['player','motion','spatial','temporal','route'],
                                 mode='fast',
                                 dtype='float32',
                                 save_dir='processed_features_fast')

**✅ 2. Full + float64 + chunked**

In [ ]:
features = add_selected_features(df_input,
                                 feature_groups=['player','motion','spatial','temporal','interaction','route'],
                                 mode='full',
                                 dtype='float64',
                                 chunked=True,
                                 save_dir='processed_features_full')


**✅ 3. Only motion & temporal features (for quick baseline)**

In [ ]:
features = add_selected_features(df_input,
                                 feature_groups=['motion','temporal'],
                                 mode='fast')
